In [12]:
import pandas as pd
import numpy as np
import time
import random
import lightgbm as lgb
import time
from multiprocessing import Process
import pickle
import matplotlib.pyplot as plt

In [2]:
def map_pred(df_pred_idx, df_map_idx, m, target_cols='predict'):
    ids = df_pred_idx.loc[m]['id'].values
    preds = df_pred_idx.loc[m]['predict'].values
    pred_dict = {ids[i]:preds[i] for i in range(ids.shape[0])}
    pred_dict[-1] = 0
    df_sc_train = df_map_idx.loc[m][df_map_idx.columns[:-1]].replace(pred_dict)
    df_sc_train['id'] = df_map_idx.loc[m]['id']
    return df_sc_train.reset_index(drop=True)

In [3]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'
TEMP = OUTPUT + 'temp/'
MAP  = OUTPUT + 'map/'

In [4]:
# original input
df_train = pd.read_csv(FOLDER+'train.csv')
df_test = pd.read_csv(FOLDER+'test.csv')

In [5]:
DATA_DATE = '20190721'
VER = '02'
MAP_DATE = '20190728'
MODEL_DATE = '20190804'
num_rep = 2

In [6]:
# fixed input
map_train = MAP + MAP_DATE + '_' + 'map_train.pickle'
map_test = MAP + MAP_DATE + '_' + 'map_test.pickle'
dist_train = MAP + MAP_DATE + '_' + 'dist_train.pickle'
dist_test = MAP + MAP_DATE + '_' + 'dist_test.pickle'

first_train = OUTPUT + DATA_DATE + '_' + 'dist_ang_ori_bond_cos_train_{}.pickle'#.format(b)
first_test = OUTPUT + DATA_DATE + '_' + 'dist_ang_ori_bond_cos_test_{}.pickle'#.format(b)

In [7]:
# input files
pred_train = OUTPUT + MODEL_DATE + '_' + VER + '_' + 'pred_train_{}.pickle'#.format(num_rep)
pred_test = OUTPUT + MODEL_DATE + '_' + VER + '_' + 'submission_{}.csv'#.format(num_rep)

In [8]:
# output files
pred_dist_train = OUTPUT + MODEL_DATE + '_' + VER + '_' + 'pred_dist_train.pickle'
pred_dist_test = OUTPUT + MODEL_DATE + '_' + VER + '_' + 'pred_dist_test.pickle'

features_train = OUTPUT + MODEL_DATE + '_' + VER + '_' + 'features_train_{}.pickle'#.format(b)
features_test = OUTPUT + MODEL_DATE + '_' + VER + '_' + 'features_test_{}.pickle'#.format(b)

trained_models = OUTPUT + MAP_DATE + '_' + VER + '_' + 'champs_models_lgb_{}_{}.pickle'#.format(b, num_rep)

pred_train_temp = TEMP + 'pred_train_{}.pickle'
pred_test_temp = TEMP + 'submission_{}.pickle'

In [9]:
rounds = 20000
params = {
        'task' : 'train',
        'boosting_type' : 'gbdt',
        'objective' : 'regression',
        'metric' : {'l1'},
        'num_leaves' : 15,
        'learning_rate' : 0.1,
        'feature_fraction' : 0.7,
    'seed':0
}

In [10]:
bond_types = ['3JHN', '3JHC','1JHC', '2JHH', '1JHN', '2JHN', '2JHC', '3JHH']

In [ ]:
num_div = 4
temp_no = 0

In [13]:
# map train-data
def map_train_data(temp_no):
    print('map train-data')
    df_map_pred_train = pd.DataFrame()
    # for m in mols_train[:10]:
    div = len(mols_train) // num_div

    # for m in mols_train[:10]:
    if temp_no == num_div - 1:
        last_slice = -1
    else:
        last_slice = div*(temp_no+1)

    for m in mols_train[div*temp_no:last_slice]:
        if m == mols_train[2]:
            continue
        df_map_pred_train_temp = map_pred(df_pred_train_idx, df_map_train_idx, m)
        df_map_pred_train = pd.concat([df_map_pred_train, df_map_pred_train_temp], axis=0)

        m = mols_train[2]
        se_temp = pd.Series()
    #     se_temp['molecule_name']  = m
        se_temp['0'] = df_pred_train_idx.loc[m]['predict']
        for i in range(48):
            se_temp['{}'.format(i+1)] = 0.0
        cols = df_map_pred_train_temp.columns[:-1]
        df_map_pred_train_temp = pd.DataFrame(se_temp).T
        df_map_pred_train_temp.columns = cols
        df_map_pred_train_temp['id'] = df_pred_train_idx.loc[m]['id']
        df_map_pred_train = pd.concat([df_map_pred_train, df_map_pred_train_temp], axis=0, sort=False)

    return pd.merge(df_map_pred_train, df_dist_train_idx, on=['id'])
#     df_pred_dist_train.to_pickle(pred_dist_train)

In [15]:
p = Process(target=map_train_data, args=(0,))
p.start

<bound method BaseProcess.start of <Process(Process-2, initial)>>

In [ ]:
for num_rep in range(10):
    print('repeat number:', num_rep)
# prepare data
    mols_train = df_train['molecule_name'].unique()
    df_pred_train = pd.read_pickle(pred_train.format(num_rep))
    df_pred_train = df_pred_train.rename(columns={'scalar_coupling_constant':'predict'})
    df_pred_train_idx = pd.merge(df_train, df_pred_train, on=['id']).set_index('molecule_name')
    df_map_train_idx = pd.read_pickle(map_train).set_index('molecule_name')
    df_dist_train_idx = pd.read_pickle(dist_train).set_index('molecule_name')

    # map train-data
    print('map train-data')
    df_map_pred_train = pd.DataFrame()
    # for m in mols_train[:10]:
    for m in mols_train:
        if m == mols_train[2]:
            continue
        df_map_pred_train_temp = map_pred(df_pred_train_idx, df_map_train_idx, m)
        df_map_pred_train = pd.concat([df_map_pred_train, df_map_pred_train_temp], axis=0)

        m = mols_train[2]
        se_temp = pd.Series()
    #     se_temp['molecule_name']  = m
        se_temp['0'] = df_pred_train_idx.loc[m]['predict']
        for i in range(48):
            se_temp['{}'.format(i+1)] = 0.0
        cols = df_map_pred_train_temp.columns[:-1]
        df_map_pred_train_temp = pd.DataFrame(se_temp).T
        df_map_pred_train_temp.columns = cols
        df_map_pred_train_temp['id'] = df_pred_train_idx.loc[m]['id']
        df_map_pred_train = pd.concat([df_map_pred_train, df_map_pred_train_temp], axis=0, sort=False)

    df_pred_dist_train = pd.merge(df_map_pred_train, df_dist_train_idx, on=['id'])
    df_pred_dist_train.to_pickle(pred_dist_train)
    del df_map_pred_train, df_dist_train_idx, df_pred_train_idx, df_map_train_idx
    gc.collect()

In [11]:
for num_rep in range(10):
    print('repeat number:', num_rep)
# prepare data
    mols_train = df_train['molecule_name'].unique()
    df_pred_train = pd.read_pickle(pred_train.format(num_rep))
    df_pred_train = df_pred_train.rename(columns={'scalar_coupling_constant':'predict'})
    df_pred_train_idx = pd.merge(df_train, df_pred_train, on=['id']).set_index('molecule_name')
    df_map_train_idx = pd.read_pickle(map_train).set_index('molecule_name')
    df_dist_train_idx = pd.read_pickle(dist_train).set_index('molecule_name')

    # map train-data
    print('map train-data')
    df_map_pred_train = pd.DataFrame()
    # for m in mols_train[:10]:
    for m in mols_train:
        if m == mols_train[2]:
            continue
        df_map_pred_train_temp = map_pred(df_pred_train_idx, df_map_train_idx, m)
        df_map_pred_train = pd.concat([df_map_pred_train, df_map_pred_train_temp], axis=0)

        m = mols_train[2]
        se_temp = pd.Series()
    #     se_temp['molecule_name']  = m
        se_temp['0'] = df_pred_train_idx.loc[m]['predict']
        for i in range(48):
            se_temp['{}'.format(i+1)] = 0.0
        cols = df_map_pred_train_temp.columns[:-1]
        df_map_pred_train_temp = pd.DataFrame(se_temp).T
        df_map_pred_train_temp.columns = cols
        df_map_pred_train_temp['id'] = df_pred_train_idx.loc[m]['id']
        df_map_pred_train = pd.concat([df_map_pred_train, df_map_pred_train_temp], axis=0, sort=False)

    df_pred_dist_train = pd.merge(df_map_pred_train, df_dist_train_idx, on=['id'])
    df_pred_dist_train.to_pickle(pred_dist_train)
    del df_map_pred_train, df_dist_train_idx, df_pred_train_idx, df_map_train_idx
    gc.collect()

    print('merge features_train')
    for b in bond_types:
        df_train_b = pd.read_pickle(first_train.format(b))
        df_feature_train = pd.merge(df_train_b, df_pred_dist_train, on=['id'])
        df_feature_train.to_pickle(features_train.format(b))
        del df_feature_train, df_train_b
        gc.collect()

    print('train models')
    scores = []
    with open(mols_split, 'rb') as f:
        mols_train = pickle.load(f)
        mols_val =  pickle.load(f)
    for b in bond_types:
        print('-'*10, b, '-'*10)
        start = time.time()

        df = pd.read_pickle(features_train.format(b)).fillna(0)

        df_train = df[df['molecule_name'].isin(mols_train)]    
        y_train = df_train['scalar_coupling_constant'].values
        X_train = df_train[df_train.columns[6:]].values
        lgb_train = lgb.Dataset(X_train, y_train)

        df_val = df[df['molecule_name'].isin(mols_val)]
        y_val = df_val['scalar_coupling_constant'].values
        X_val = df_val[df_val.columns[6:]].values
        lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

        gbm = lgb.train(params,
                lgb_train,
                num_boost_round=rounds,
                verbose_eval=0,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)

        with open(trained_models.format(b, num_rep+1), 'wb') as f:
            pickle.dump(gbm, f)

        y_val_pred = gbm.predict(X_val, num_iteration=gbm.best_iteration)
        y_train_pred = gbm.predict(X_train, num_iteration=gbm.best_iteration)

        df_pred_train = pd.DataFrame([df_train['id'], y_pred_train], index=['id', 'scalar_coupling_constant']).T
        df_pred_train['id'] = df_pred_train['id'].astype('int32')
        df_pred_test = pd.DataFrame([df_test['id'], df_pred_test], index=['id', 'scalar_coupling_constant']).T
        df_pred_test['id'] = df_pred_test['id'].astype('int32')    

        df_pred = pd.concat([df_pred_train, df_pred_test], axis=0, sort=False).sort_values('id').reset_index(drop=True)
        df_pred.to_pickle(pred_train_temp.format(b))

        score_first_train = type_score(y_train, X_train[:, -98])
        score_second_train = type_score(y_train, y_train_pred)

        score_first = type_score(y_val, X_val[:, -98])
        score_second = type_score(y_val, y_val_pred)
        scores.append(score_second)
        print(gbm.best_iteration)
        print("score first model(train): %.2f" %  (score_first_train))
        print("score second model(train): %.2f" %  (score_second_train))
        print("score first model(val): %.2f" %  (score_first))
        print("score second model(val): %.2f" %  (score_second))
        elapsed_time = time.time() - start
        print ("elapsed_time:%.2f" % elapsed_time + "[sec]")
    print('total score:', np.array(scores).mean())

    print('merge prediction')    
    df_pred_train = pd.DataFrame()
    for b in bond_types:
        df_b = pd.read_pickle(pred_train_temp.format(b))
        df_pred_train = pd.concat([df_pred_train, df_b], axis=0, sort=False)

    df_pred_train['id'] = df_pred_train['id'].astype('int32')
    df_pred_train = df_pred_train.reset_index(drop=True)
    df_pred_train.to_pickle(pred_train.format(num_rep+1))
    del df_pred_train, df_b
    gc.collect()


    # prepara test data
    mols_test = df_test['molecule_name'].unique()
    df_pred_test = pd.read_csv(pred_test.format(num_rep))
    df_pred_test = df_pred_test.rename(columns={'scalar_coupling_constant':'predict'})
    df_pred_test_idx = pd.merge(df_test, df_pred_test, on=['id']).set_index('molecule_name')
    df_map_test_idx = pd.read_pickle(map_test).set_index('molecule_name')
    df_dist_test_idx = pd.read_pickle(dist_test).set_index('molecule_name')

    # map test-data
    print('map test-data')
    df_map_pred_test = pd.DataFrame()
    # for m in mols_test[:10]:
    for m in mols_test:
        df_map_pred_test_temp = map_pred(df_pred_test_idx, df_map_test_idx, m)
        df_map_pred_test = pd.concat([df_map_pred_test, df_map_pred_test_temp], axis=0, sort=False)
    df_pred_dist_test = pd.merge(df_map_pred_test, df_dist_test_idx, on=['id'])
    df_pred_dist_test.to_pickle(pred_dist_test)
    del df_map_pred_test, df_map_pred_test_temp
    gc.collect()

    print('merge features_test')
    for b in bond_types:
        df_test_b = pd.read_pickle(first_test.format(b))
        df_feature_test = pd.merge(df_test_b, df_pred_dist_test, on=['id'])
        df_feature_test.to_pickle(features_test.format(b))
        del df_feature_test, df_test_b
        gc.collect()
    del df_pred_dist_test
    gc.collect()

    print('get test prediction')
    for b in bond_types:
        print('-'*10, b, '-'*10)
        start = time.time()
        df_bond = pd.read_pickle(features_test.format(b))
        y_pred_b =[]

        with open(trained_models.format(b, num_rep+1), 'rb') as f:
            gbm = pickle.load(f)

        X_test = df_bond[df_bond.columns[5:]].values  
        y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

        df_pred = pd.DataFrame([df_bond['id'], y_pred], index=['id', 'scalar_coupling_constant']).T
        df_pred['id'] = df_pred['id'].astype('int32')
        df_pred.to_pickle(pred_test_temp.format(b))
        del df_pred, gbm, df_bond
        gc.collect()

    df_submit = pd.DataFrame()
    for b in bond_types:
        df_submit_b = pd.read_pickle(pred_test_temp.format(b))
        df_submit = pd.concat([df_submit, df_submit_b], axis=0)

    df_submit['id'] = df_submit['id'].astype('int32')
    df_submit = df_submit.sort_values('id').reset_index(drop=True)
    df_submit.to_csv(pred_test.format(num_rep+1), index=False)
    del df_submit, df_submit_b
    gc.collect()

repeat number: 0
map train-data


KeyboardInterrupt: 

In [ ]:
# (4658147, 100) (2505542, 100)